In [ ]:
from scipy.stats.distributions import gamma
import matplotlib.pyplot as plt
import dautil as dl
import pandas as pd
from IPython.display import HTML

In [ ]:
lr = dl.nb.LatexRenderer(chapter=3, start=2)
lr.render(r'f(x;k,\theta) =  \frac{x^{k-1}e^{-\frac{x}{\theta}}}{\theta^k\Gamma(k)} \quad \text{ for } x > 0 \text{ and } k, \theta > 0')
lr.render(r'\scriptstyle \mathbf{E}[ X] = k \theta')
lr.render(r'\scriptstyle\operatorname{Var}[ X] = k \theta^2')

In [ ]:
rain = dl.data.Weather.load()['RAIN'].resample('M').dropna()
rain = dl.ts.groupby_month(rain)
rain = rain.get_group(1)

dist = dl.stats.Distribution(rain, gamma)

a = (dist.mean() ** 2)/dist.var()
shape, loc, scale = dist.fit(a)

result = dl.report.HTMLBuilder()
result.h1('Fitting Data to the Gamma Distribution')
table = dl.report.DFBuilder(['loc', 'scale', 'shape'])
table.row([loc, scale, shape])
dl.options.set_pd_options()
result.h2('Distribution Parameters')
result.add_df(table.build())

result.h2('Residuals of the Fit')
table2 = dist.describe_residuals(a=shape, loc=loc, scale=scale)
result.add(table2.to_html())

result.h2('Fit Metrics')
table3 = pd.DataFrame([{'RMSE': dist.rmse(), 'Mean_AD': dist.mean_ad()}])
result.add_df(table3)

In [ ]:
%matplotlib inline
dl.options.mimic_seaborn()
context = dl.nb.Context('fitting_gamma')
dl.nb.RcWidget(context)

In [ ]:
fig, ax = plt.subplots()
plt.title('Fitting to the gamma distribution')

# Limiting the x-asis for a better plot
plt.xlim([0, 15])
plt.xlabel(dl.data.Weather.get_header('RAIN'))
dist.plot(ax)
HTML(result.html)